In [58]:
%pylab inline
from snakebite.client import Client
import pyaml as yaml
import sys
import pythonrun
reload(pythonrun)
import os
import seaborn as sns
from sklearn import metrics
import scala_python_endive_wrapper
reload(scala_python_endive_wrapper)
import score
from scala_python_endive_wrapper import *
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


In [39]:
hdfsclient = Client("amp-bdg-master.amplab.net", 8020, use_trash=False)
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')
logpath = "/home/eecs/akmorrow/ADAM/endive"
predicted_output = "hdfs://amp-bdg-master.amplab.net:8020/user/akmorrow/predictions/"


In [40]:
executor_mem = "100g"
cores_per_executor = 8
num_executors = 50
EGR1_PATH = "/data/anv/DREAMDATA/aggregated/full_dnase/EGR1"

featurized = "hdfs://amp-spark-master.amp:8020/user/akmorrow/featurized/EGR1_noNormalizedDnase_dim_2048_samp_0.01"

In [41]:
# chromosomes = map(lambda x: "chr{0}".format(x+1), range(23))
# chromosomes.append("chrX")
chromosomes=["chr15"]
# CHANGE ME BASED ON ENUM + TF
egr1_cell_types = string_to_enum_celltypes(['GM12878', 'H1hESC', 'HCT116', 'MCF7'])

In [54]:
reload(scala_python_endive_wrapper)
results_df = cross_validate(featurized, 
                            hdfsclient, 
                            chromosomes, 
                            egr1_cell_types,
                            logpath,
                            numHoldOutChr=1, 
                            numHoldOutCell=1,
                            num_folds=1, 
                            negativeSamplingFreqs=[0.01],
                            mixtureWeights=[-1.0],
                            cores_per_executor=8,
                            num_executors=16,
                            regs=[1e10])

FOLD 0
HOLDING OUT CHROMOSOMES ['chr15']
HOLDING OUT CELL TYPES [3]
RUNING SOLVER WITH REG=10000000000.0
AND MIXTUREWEIGHT=-1.0


KeyboardInterrupt: 

In [30]:
results_df

,negativeSamplingFreq,reg,test_celltypes,test_chromosomes,train_auPRC,train_auROC,train_recall_at_05_fdr,train_recall_at_10_fdr,train_recall_at_25_fdr,train_recall_at_50_fdr,val_auPRC,val_auROC,val_recall_at_05_fdr,val_recall_at_10_fdr,val_recall_at_25_fdr,val_recall_at_50_fdr
0,0.1,1.000000e+10,[3],[chr15],0.349011,0.9769,0.0,0.0,0.0,0.044546,0.108895,0.945704,0.0,0.0,0.0,0.0


In [ ]:
predictionsPath = "/home/eecs/akmorrow/tmp/L.EGR1.K562.tab"
modelPath = "/home/eecs/akmorrow/endive-models-2048"
featuresPath = "hdfs://amp-spark-master.amp:8020/user/akmorrow/testFeatures/K562_dim_2048_dim_2048_samp_1.0"
approxDim = 2048

(test_preds, meta_df) = run_test_pipeline(featuresPath, # features for test windows
               logpath,
               hdfsclient,
               predictionsPath, # saves metadata and test preditions to /testMetaData and /testPreds
               modelPath, 
               approxDim)

{'predictionsOutput': '/home/eecs/akmorrow/tmp/L.EGR1.K562.tab', 'reference': '/home/eecs/akmorrow/ADAM/endive/workfiles/hg19.2bit', 'dnaseBams': '/data/anv/DREAMDATA/dnase_bams/merged_coverage/', 'featuresOutput': 'hdfs://amp-spark-master.amp:8020/user/akmorrow/testFeatures/K562_dim_2048_dim_2048_samp_1.0', 'dnaseNarrow': '/data/anv/DREAMDATA/DNASE/peaks/relaxed/', 'approxDim': 2048, 'ladderBoard': True, 'modelOutput': '/home/eecs/akmorrow/endive-models-2048'}


In [67]:
reload(score)
truePath = "/home/eecs/akmorrow/tmp/ladder_regions.blacklistfiltered.bed"
predictionsPath = "/home/eecs/akmorrow/tmp/L.EGR1.K562.tab"
score.verify_file_and_build_scores_array(truePath, predictionsPath, 3)


chr1	650	850	0.0



InputError: Line 1 in submitted file does not match line 1 in the reference regions file